First we load the required libraries

In [33]:
import ee
from IPython.display import Image

ee.Initialize()

## First example: elevation data

Note for each dataset the urls inside ```ee.Image``` can be obtained from https://developers.google.com/earth-engine/datasets

In [34]:
# Access to elevation data
im = ee.Image('USGS/SRTMGL1_003')

# We can specify a particular point in the map
xy = ee.Geometry.Point([86.9250, 27.9881])

# We sample one "pixel" value and obtain the elevation feature
elev = im.sample(xy, 1).first().get('elevation').getInfo()
print('Mount Everest elevation (m):', elev)

Mount Everest elevation (m): 8729


Next, instead of focusing on a single point, we will plot all the elevation over the Earth, using a custom palette

In [35]:
Image(url = im.updateMask(im.gt(0))  # This is to filter out sub-zero elevation (oceans mostly)
  .getThumbUrl({'min': 0, 'max': 4000, 'dimensions': 1024,
                'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}))

## Second example: illumination data

Note that for this dataset we have a collection of images instead of a single ```Image```, so we need to use a reduction operation such as mean, median, std, max... to aggregate to a single image

![collection](img/collection.png)

In this example we will compute the mean illumination over all the time-periods

In [36]:
collection = ee.ImageCollection("NOAA/DMSP-OLS/NIGHTTIME_LIGHTS")

collection = collection.select('stable_lights')
collection = collection.reduce(ee.Reducer.mean());

In [37]:
Image(url=collection.getThumbUrl({'min': 0, 'max': 0.18, 'dimensions': 2048,
                'palette': ['black', 'cyan']}))

Now we will plot a more interesting example using this Reducer https://developers.google.com/earth-engine/reducers_regression

In [38]:
collection = ee.ImageCollection("NOAA/DMSP-OLS/NIGHTTIME_LIGHTS")

# Add a band containing image date as years since 1991.
def CreateTimeBand(img):
    year = ee.Date(img.get('system:time_start')).get('year').subtract(1991)
    return ee.Image(year).byte().addBands(img)

collection = collection.select('stable_lights').map(CreateTimeBand)
collection = collection.reduce(ee.Reducer.linearFit());

Using an RGB color pallette, if the slope of the regression is possitive (illumination has increased during the years), the pixel will be more red. If the slope is negative, the pixel will be bluer. And if the illumination has been approximately constant, the color will be green

In [39]:
Image(url=collection.getThumbUrl({'min': 0, 'max': '0.18,20,-0.18',
      'bands': 'scale,offset,scale', 'dimensions': 2048}))

To get dynamic plots (using folium), see the next notebook, where we plot pollution examples